In [2]:
from reweighing import weighting_educ, weighting_maritalstatus

In [73]:
# applying weights
import csv
gender = []
educ = []
status = []
with open('adult.csv', newline='') as data:
    spamreader = csv.reader(data, delimiter=',')
    for row in spamreader:
        #[1:-1] because the words look like this: "'word'"
        educ.append(row[3][1:-1])
        gender.append(row[9][1:-1])
        status.append(row[5][1:-1])
pos_level, male_educ, female_educ = weighting_educ(gender, educ)
pos_status, male_mar, female_mar = weighting_maritalstatus(gender, status)
print(pos_level, "\n", male_educ,"\n" , female_educ, "\n", pos_status, "\n", male_mar, "\n", female_mar)

dict_keys(['8', '11', 'HS-grad', 'Some-college', 'assoc', 'Bachelors', 'Masters', 'Prof-school', 'Doctorate']) 
 [0.2269792427365061, 0.3093568656412059, 0.6694595918070898, 0.5161013981243859, 0.9998019359098849, 0.9703584627146346, 1.2312560881987997, 0.5530240269659172, 1.163563571096275] 
 [0.47621976555833023, 0.6203936279326979, 1.3322053589817218, 0.8667132892975913, 2.040141036814523, 2.475720131227805, 3.684885360226981, 2.4776717415707235, 3.6863151712957185] 
 dict_keys(['Never-married', 'Married', 'Separated', 'Divorced', 'Widowed']) 
 [0.9930934758755159, 0.5787769230260589, 0.6427218448608766, 0.8536773849373384, 2.494169552848252] 
 [1.0741509227484345, 4.144875173611973, 0.574172801665185, 0.7574571502995334, 1.7530552344190702]
